In [22]:
# Developed by Humza, Afnan, Saad, Ahsan, Nabeel

import pandas as pd
import os
import glob
import numpy as np
from twarc import Twarc

In [2]:
# Use this code to take a sample of tweets for hydration

list_dir = ['2020-09_og']
final_ids = dict()

for direct in list_dir:
    n_files = list()
    path_dir = os.getcwd() + "\\" + direct
    og_files = glob.glob(path_dir + "/*.txt")
    for file in og_files:
        ids = pd.read_csv(file,header=None)
        sample = ids.iloc[:,0].sample(n=40000,random_state=1,replace=False)
        sample = list(sample)
        n_files.append(sample)
    n_files = np.concatenate(n_files)
    final_ids[direct] = n_files

for direct in final_ids.keys():
    new_dir = direct.split("_")[0] 
    os.mkdir(new_dir)
    path_upd = os.getcwd() + "\\" + new_dir
    if '-09' in direct:
        np.savetxt((path_upd + '\\pre_pres_deb1.txt'),final_ids[direct],fmt='%d')
    elif '-10' in direct:
        np.savetxt((path_upd + '\\post_pres_deb1.txt'),final_ids[direct],fmt='%d')

In [36]:
#
# This script will walk through all the tweet id files and
# hydrate them with twarc. The line oriented JSON files will
# be placed right next to each tweet id file.
#
# Note: you will need to install twarc, tqdm, and run twarc configure
# from the command line to tell it your Twitter API keys.
#
# Special thanks to Github users edsu and SamSamhuns for contributing to this file. This file was repurposed from our other
# data repository on COVID-19 related tweets : https://github.com/echen102/COVID-19-TweetIDs
#

import gzip
import json
import pandas as pd

from tqdm import tqdm
from twarc import Twarc
from pathlib import Path

twarc = Twarc()
data_dirs = ['BLM_test'] # can give multiple directories if you have made tweet id files for separate months. follow the naming scheme i.e. YYYY-MM

def main():
    for data_dir in data_dirs:
        for path in Path(data_dir).iterdir():
            if path.name.endswith('.txt'):
                hydrate(path)


def _reader_generator(reader):
    b = reader(1024 * 1024)
    while b:
        yield b
        b = reader(1024 * 1024)


def raw_newline_count(fname):
    """
    Counts number of lines in file
    """
    f = open(fname, 'rb')
    f_gen = _reader_generator(f.raw.read)
    return sum(buf.count(b'\n') for buf in f_gen)


def hydrate(id_file):
    print('hydrating {}'.format(id_file))

    gzip_path = id_file.with_suffix('.jsonl.gz')
    if gzip_path.is_file():
        print('skipping json file already exists: {}'.format(gzip_path))
        return

    num_ids = raw_newline_count(id_file)

    with gzip.open(gzip_path, 'w') as output:
        with tqdm(total=num_ids) as pbar:
            for tweet in twarc.hydrate(id_file.open()):
                output.write(json.dumps(tweet).encode('utf8') + b"\n")
                pbar.update(1)


if __name__ == "__main__":
    main()

  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]

hydrating BLM_test\test.txt


 14%|███████████                                                                  | 1434/10000 [00:54<05:23, 26.50it/s]


In [6]:
# Run this after running the second cell which hydrated the ids. Use the directory named based on folder name where you stored id file. 
# Format for directory is YYYY-MM

import os
import glob
import pandas as pd

data_dirs = ['2020-09']
hyd_tweets = dict()

for directory in data_dirs:
    path = os.getcwd() + f'\\{directory}\\'
    all_files = glob.glob(path + "/*.gz")
    li = []
    for filename in all_files:
        df = pd.read_json(filename,compression='infer',lines=True)
        li.append(df)
    data = pd.concat(li, axis=0, ignore_index=True)
    hyd_tweets[directory] = data
    data.to_csv(path + directory + '.csv')
    
# finally files created for exploration